<a href="https://colab.research.google.com/github/baranouskiart/rmf_analysis/blob/main/rmf_analysis_by_invoice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import warnings

#Load Data

In [2]:
df=pd.read_excel('/content/Online Retail.xlsx') #I will change this placeholder for another from BiqQuerry orig after data collection process
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df.info() #I see the null numbers in description and CustomerID

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [4]:
df.isnull().sum()
#why there are invoices w/t customerID

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [5]:
df.isnull().sum()*100/len(df)
#what is percent of invoices w/t customers

InvoiceNo       0.000000
StockCode       0.000000
Description     0.268311
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
CustomerID     24.926694
Country         0.000000
dtype: float64

## Data Preprocessing

In [10]:
df.drop('InvoiceDate', axis=1).describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00,10.00,80995.0
UnitPrice,541909.0,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0
CustomerID,406829.0,15287.690570,1713.600303,12346.00,13953.00,15152.00,16791.00,18287.0


In [11]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [18]:
duplicates = df['InvoiceNo'].duplicated()
duplicates.sum()

516009

In [14]:
df['Total']=df['Quantity']*df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [13]:
df.groupby(['InvoiceNo'])['Total'].sum().sort_values()
 #chech the summarization of the similar invoices
 #negatiive sums could show that order was cancelled - C

InvoiceNo
C581484   -168469.60
C541433    -77183.60
C556445    -38970.00
C550456    -22998.40
C580605    -17836.46
             ...    
556444      38970.00
576365      50653.91
574941      52940.94
541431      77183.60
581483     168469.60
Name: Total, Length: 25900, dtype: float64

In [20]:
df.loc[df['Total']<0]['Total'].sum().round() #total of the canceled orders
#from this, we can say that  if the new 'Total' column  we created  is less than zero,
#that order has probably been canceled or abandoned.

-918937.0

In [21]:
df.loc[df['Total']<0].groupby(['InvoiceNo'])[['Total']].sum()
#need to clarify the reason of duplicated values in AC docs

,Total
InvoiceNo,
A563186,-11062.06
A563187,-11062.06
C536379,-27.50
C536383,-4.65
C536391,-141.48
...,...
C581484,-168469.60
C581490,-32.53
C581499,-224.69


In [22]:
len(df.loc[df['Quantity']<0])
#the number of invoices with minus values orders

10624